In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
#leer la bd
df = pd.read_csv("../../data/processed/BD_ID_CLIENTE_N.POLIZA_X_MES.csv", index_col="Unnamed: 0")
df.head()

,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01
1690,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9860,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [3]:
#agrego la bd de datos cliente
df_cli = pd.read_csv("../../data/processed/BD_ID_CLIENTE_D.Pers_Modo.Pago.csv", index_col="Unnamed: 0")
df_cli.set_index("CUSTOMER_ID", inplace=True)
df_cli.sort_index(inplace=True)
df_cli.head()

,DATE_OF_BIRTH,GENDER,MARITAL_STATUS,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad,ModoPago,edad_r
CUSTOMER_ID,,,,,,,,,,,,
1690,1943-03-06,M,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
5575,1966-06-10,M,Casado,NaN,NaN,ESCB,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2017,51.0,Mix,4.0
7968,1911-01-11,M,Casado,NaN,NaN,NaN,NaN,NaN,31/01/2018,107.0,CC,0.0
9028,1961-07-15,M,Casado,NaN,NaN,ARQU,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2018,56.0,Mix,4.0
9860,1967-04-06,M,Soltero,NaN,NaN,EMPL,CAPITAL FEDERAL,CAPITAL FEDERAL,31/01/2016,50.0,Mix,4.0


# armando bloques 9 m (6 train, 3 target)

### Creo lista de dataframes

In [4]:
df = df.iloc[:,:-2]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 1690 to 21260067
Columns: 165 entries, 2004-03 to 2017-11
dtypes: float64(165)
memory usage: 63.3 MB


In [7]:
len(df.columns) * 0.8

132.0

###  Genero 2 df (train y test)
80/20

In [11]:
df_train = df.iloc[:,:133]
df_train.head()

,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,...,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,2014-12,2015-01,2015-02,2015-03
1690,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0
9860,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_test = df.iloc[:,133:]
df_test.head()

,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,...,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11
1690,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN


In [13]:
#integro train
#termino en 5 para que no de error en el rango de las columnas
list_df_train = list()
for i in range( (len(df_train.columns)) ,8, -1):
    #print("{}-{}".format(i-9,i))
    list_df_train.append(df_train.iloc[:,i-9:i])
len(list_df_train)

125

In [14]:
#integro test
#termino en 5 para que no de error en el rango de las columnas
list_df_test = list()
for i in range( (len(df_test.columns)) ,8, -1):
    #print("{}-{}".format(i-9,i))
    list_df_test.append(df_test.iloc[:,i-9:i])
len(list_df_test)

24

### Renombre las columnas de cada dataframe

In [15]:
#cambiando el nombre de las fechas a versión generica ( 6 meses train, 3 pred)
cols = ["6m", "5m", "4m", "3m", "2m", "1m", "3p", "2p", "1p"]

#train
for i in range(0, len(list_df_train)):
    list_df_train[i].rename(columns=dict(zip(list_df_train[i].columns[:9], cols)),inplace=True)
#test
for i in range(0, len(list_df_test)):
    list_df_test[i].rename(columns=dict(zip(list_df_test[i].columns[:9], cols)),inplace=True)

/home/seba/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


### Integrando lista de dataframes en un único dataframe

In [16]:
bd_train = pd.concat(list_df_train)
bd_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6250000 entries, 1690 to 21260067
Data columns (total 9 columns):
6m    float64
5m    float64
4m    float64
3m    float64
2m    float64
1m    float64
3p    float64
2p    float64
1p    float64
dtypes: float64(9)
memory usage: 476.8 MB


In [17]:
bd_test = pd.concat(list_df_test)
bd_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200000 entries, 1690 to 21260067
Data columns (total 9 columns):
6m    927968 non-null float64
5m    923151 non-null float64
4m    917812 non-null float64
3m    911914 non-null float64
2m    905460 non-null float64
1m    898548 non-null float64
3p    891143 non-null float64
2p    883006 non-null float64
1p    874053 non-null float64
dtypes: float64(9)
memory usage: 91.6 MB


### Integrando con dataframe clientes

In [18]:
df_int_train = pd.merge(bd_train, df_cli, left_index=True, right_index=True)
df_int_train.info()

df_int_test = pd.merge(bd_test, df_cli, left_index=True, right_index=True)
df_int_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6250000 entries, 1690 to 21260067
Data columns (total 21 columns):
6m                 float64
5m                 float64
4m                 float64
3m                 float64
2m                 float64
1m                 float64
3p                 float64
2p                 float64
1p                 float64
DATE_OF_BIRTH      object
GENDER             object
MARITAL_STATUS     object
EDUCATION          object
EDUCATION_LEVEL    object
OCCUPATION         object
CITY               object
LOCALITY           object
FECHA_PROCESO      object
edad               float64
ModoPago           object
edad_r             float64
dtypes: float64(11), object(10)
memory usage: 1.0+ GB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200000 entries, 1690 to 21260067
Data columns (total 21 columns):
6m                 927968 non-null float64
5m                 923151 non-null float64
4m                 917812 non-null float64
3m                 911914 

In [19]:
df_int_train.drop_duplicates(inplace=True)
df_int_test.drop_duplicates(inplace=True)
df_int_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626883 entries, 1690 to 21260067
Data columns (total 21 columns):
6m                 271012 non-null float64
5m                 296497 non-null float64
4m                 323500 non-null float64
3m                 351089 non-null float64
2m                 378931 non-null float64
1m                 406950 non-null float64
3p                 435243 non-null float64
2p                 463852 non-null float64
1p                 492317 non-null float64
DATE_OF_BIRTH      621633 non-null object
GENDER             626544 non-null object
MARITAL_STATUS     582386 non-null object
EDUCATION          212114 non-null object
EDUCATION_LEVEL    21294 non-null object
OCCUPATION         390966 non-null object
CITY               468853 non-null object
LOCALITY           468853 non-null object
FECHA_PROCESO      626883 non-null object
edad               621633 non-null float64
ModoPago           626883 non-null object
edad_r             621633 non-null 

In [67]:
df_int.head()

,6m,5m,4m,3m,2m,1m,3p,2p,1p,DATE_OF_BIRTH,...,MARITAL_STATUS,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad,ModoPago,edad_r
1690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
1690,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
1690,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
1690,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0
1690,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1943-03-06,...,Casado,S,NaN,CONT,BUENOS AIRES,HURLINGHAM,31/01/2014,74.0,CC,5.0


In [20]:
#variables descartadas
delete_cols = ["DATE_OF_BIRTH", "FECHA_PROCESO", "OCCUPATION", "LOCALITY", "edad"]
df_fin_train = df_int_train.drop(delete_cols, axis=1)
df_fin_test = df_int_test.drop(delete_cols, axis=1)
df_fin_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626883 entries, 1690 to 21260067
Data columns (total 16 columns):
6m                 271012 non-null float64
5m                 296497 non-null float64
4m                 323500 non-null float64
3m                 351089 non-null float64
2m                 378931 non-null float64
1m                 406950 non-null float64
3p                 435243 non-null float64
2p                 463852 non-null float64
1p                 492317 non-null float64
GENDER             626544 non-null object
MARITAL_STATUS     582386 non-null object
EDUCATION          212114 non-null object
EDUCATION_LEVEL    21294 non-null object
CITY               468853 non-null object
ModoPago           626883 non-null object
edad_r             621633 non-null float64
dtypes: float64(10), object(6)
memory usage: 81.3+ MB


In [21]:
X_train = pd.get_dummies(df_fin_train)
X_test = pd.get_dummies(df_fin_test)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626883 entries, 1690 to 21260067
Data columns (total 58 columns):
6m                                271012 non-null float64
5m                                296497 non-null float64
4m                                323500 non-null float64
3m                                351089 non-null float64
2m                                378931 non-null float64
1m                                406950 non-null float64
3p                                435243 non-null float64
2p                                463852 non-null float64
1p                                492317 non-null float64
edad_r                            621633 non-null float64
GENDER_F                          626883 non-null uint8
GENDER_M                          626883 non-null uint8
MARITAL_STATUS_Casado             626883 non-null uint8
MARITAL_STATUS_Conviv             626883 non-null uint8
MARITAL_STATUS_Divorciado         626883 non-null uint8
MARITAL_STATUS_No Corres

In [23]:
X_train.fillna(0, inplace=True) #reemplazo nulos por 0
X_train = X_train.astype("uint8") #cambio el tipo de los datos

X_test.fillna(0, inplace=True) #reemplazo nulos por 0
X_test = X_test.astype("uint8") #cambio el tipo de los datos
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626883 entries, 1690 to 21260067
Data columns (total 58 columns):
6m                                626883 non-null uint8
5m                                626883 non-null uint8
4m                                626883 non-null uint8
3m                                626883 non-null uint8
2m                                626883 non-null uint8
1m                                626883 non-null uint8
3p                                626883 non-null uint8
2p                                626883 non-null uint8
1p                                626883 non-null uint8
edad_r                            626883 non-null uint8
GENDER_F                          626883 non-null uint8
GENDER_M                          626883 non-null uint8
MARITAL_STATUS_Casado             626883 non-null uint8
MARITAL_STATUS_Conviv             626883 non-null uint8
MARITAL_STATUS_Divorciado         626883 non-null uint8
MARITAL_STATUS_No Corresp         626883 non

In [75]:
#X["id_cliente"] = X.index

In [25]:
#X.drop_duplicates(inplace=True)
#X.info()

In [24]:
X_train.to_csv("../../data/processed/m1.0 4.3.a t - X_train - dummy sin churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")
X_test.to_csv("../../data/processed/m1.0 4.3.a t - X_test - dummy sin churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")

# Training : churn exacto 3 meses

In [26]:
df = pd.read_csv("../../data/processed/m1.0 4.3.a t - X_train - dummy sin churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv", index_col="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626883 entries, 1690 to 21260067
Data columns (total 58 columns):
6m                                626883 non-null int64
5m                                626883 non-null int64
4m                                626883 non-null int64
3m                                626883 non-null int64
2m                                626883 non-null int64
1m                                626883 non-null int64
3p                                626883 non-null int64
2p                                626883 non-null int64
1p                                626883 non-null int64
edad_r                            626883 non-null int64
GENDER_F                          626883 non-null int64
GENDER_M                          626883 non-null int64
MARITAL_STATUS_Casado             626883 non-null int64
MARITAL_STATUS_Conviv             626883 non-null int64
MARITAL_STATUS_Divorciado         626883 non-null int64
MARITAL_STATUS_No Corresp         626883 non

In [27]:
df["Churn"] = 0
#Regla churn stricta
df.loc[ (df["3p"] == 0) & (df["2p"] == 0)  &  (df["1p"] == 0) &  (df["1m"] != 0) &  (df.iloc[:,:3].count(axis=1) > 0)  &  (df.iloc[:,1:4].count(axis=1) > 0) &  (df.iloc[:,2:5].count(axis=1) > 0), "Churn"] = 1

In [28]:
df.head()

,6m,5m,4m,3m,2m,1m,3p,2p,1p,edad_r,...,CITY_TIERRA DEL FUEGO,CITY_TUCUMAN,ModoPago_BA,ModoPago_CC,ModoPago_Mix,ModoPago_PP,ModoPago_SJ,ModoPago_TA,ModoPago_TM,Churn
1690,1,1,1,1,1,1,2,1,1,5,...,0,0,0,1,0,0,0,0,0,0
1690,1,1,1,1,1,1,1,2,1,5,...,0,0,0,1,0,0,0,0,0,0
1690,1,1,1,1,1,1,1,1,2,5,...,0,0,0,1,0,0,0,0,0,0
1690,1,1,1,1,1,1,1,1,1,5,...,0,0,0,1,0,0,0,0,0,0
1690,0,1,1,1,1,1,1,1,1,5,...,0,0,0,1,0,0,0,0,0,0


In [29]:
df["Churn"].value_counts() / len(df)

0    0.980808
1    0.019192
Name: Churn, dtype: float64

In [30]:
#df.drop("id_cliente", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626883 entries, 1690 to 21260067
Data columns (total 59 columns):
6m                                626883 non-null int64
5m                                626883 non-null int64
4m                                626883 non-null int64
3m                                626883 non-null int64
2m                                626883 non-null int64
1m                                626883 non-null int64
3p                                626883 non-null int64
2p                                626883 non-null int64
1p                                626883 non-null int64
edad_r                            626883 non-null int64
GENDER_F                          626883 non-null int64
GENDER_M                          626883 non-null int64
MARITAL_STATUS_Casado             626883 non-null int64
MARITAL_STATUS_Conviv             626883 non-null int64
MARITAL_STATUS_Divorciado         626883 non-null int64
MARITAL_STATUS_No Corresp         626883 non

In [31]:
df.drop(["3p", "2p", "1p"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626883 entries, 1690 to 21260067
Data columns (total 56 columns):
6m                                626883 non-null int64
5m                                626883 non-null int64
4m                                626883 non-null int64
3m                                626883 non-null int64
2m                                626883 non-null int64
1m                                626883 non-null int64
edad_r                            626883 non-null int64
GENDER_F                          626883 non-null int64
GENDER_M                          626883 non-null int64
MARITAL_STATUS_Casado             626883 non-null int64
MARITAL_STATUS_Conviv             626883 non-null int64
MARITAL_STATUS_Divorciado         626883 non-null int64
MARITAL_STATUS_No Corresp         626883 non-null int64
MARITAL_STATUS_Separado           626883 non-null int64
MARITAL_STATUS_Soltero            626883 non-null int64
MARITAL_STATUS_Union Civ          626883 non

In [32]:
df.to_csv("../../data/processed/train/m1.0 4.3.a t - X_train -  - churn estricto - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")

## ahora con la de test

In [33]:
df = pd.read_csv("../../data/processed/m1.0 4.3.a t - X_test - dummy sin churn definido  - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv", index_col="Unnamed: 0")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261277 entries, 1690 to 21260067
Data columns (total 58 columns):
6m                                261277 non-null int64
5m                                261277 non-null int64
4m                                261277 non-null int64
3m                                261277 non-null int64
2m                                261277 non-null int64
1m                                261277 non-null int64
3p                                261277 non-null int64
2p                                261277 non-null int64
1p                                261277 non-null int64
edad_r                            261277 non-null int64
GENDER_F                          261277 non-null int64
GENDER_M                          261277 non-null int64
MARITAL_STATUS_Casado             261277 non-null int64
MARITAL_STATUS_Conviv             261277 non-null int64
MARITAL_STATUS_Divorciado         261277 non-null int64
MARITAL_STATUS_No Corresp         261277 non

In [34]:
df["Churn"] = 0
#Regla churn stricta
df.loc[ (df["3p"] == 0) & (df["2p"] == 0)  &  (df["1p"] == 0) &  (df["1m"] != 0) &  (df.iloc[:,:3].count(axis=1) > 0)  &  (df.iloc[:,1:4].count(axis=1) > 0) &  (df.iloc[:,2:5].count(axis=1) > 0), "Churn"] = 1

In [35]:
df["Churn"].value_counts() / len(df)

0    0.951328
1    0.048672
Name: Churn, dtype: float64

In [36]:
df.drop(["3p", "2p", "1p"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261277 entries, 1690 to 21260067
Data columns (total 56 columns):
6m                                261277 non-null int64
5m                                261277 non-null int64
4m                                261277 non-null int64
3m                                261277 non-null int64
2m                                261277 non-null int64
1m                                261277 non-null int64
edad_r                            261277 non-null int64
GENDER_F                          261277 non-null int64
GENDER_M                          261277 non-null int64
MARITAL_STATUS_Casado             261277 non-null int64
MARITAL_STATUS_Conviv             261277 non-null int64
MARITAL_STATUS_Divorciado         261277 non-null int64
MARITAL_STATUS_No Corresp         261277 non-null int64
MARITAL_STATUS_Separado           261277 non-null int64
MARITAL_STATUS_Soltero            261277 non-null int64
MARITAL_STATUS_Union Civ          261277 non

In [37]:
df.to_csv("../../data/processed/train/m1.0 4.3.a t - X_test - churn estricto - 6m.N.polizaXmes y Info.Cliente.M.Pago.csv")